# 总览

## 1.城市间，无航班，最低价格列表(CtoC_min_price)
## 2.城市间航班，历史价格列表(CtoC_Flight_price)

# 下载数据

In [ ]:
import findspark
import time
from pyspark.sql import SQLContext, SparkSession

findspark.init()
spark = SparkSession.Builder().appName('sql').master('local').getOrCreate()

In [ ]:
prop = {
    'user':'Zxy3',
    'password':'3.14159zsyzl',
    'driver':'com.mysql.jdbc.Driver'
}
url = 'jdbc:mysql://106.54.70.122:3306/flight'

In [ ]:
df = spark.read.jdbc(url=url, table='flightpricerawdata', properties=prop)
print('connect success')

In [ ]:
df.write.json("file:///g:/json")

# 数据处理

In [10]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.types import IntegerType
import json

In [2]:
sparkSession = SparkSession.builder.getOrCreate()
df = sparkSession.read.json("./final_min_price_list.json") # 即flightpricerawdata表
df.show(3)

+------+------+--------------+--------------------+
|arr_ct|dep_ct|          line|          price_list|
+------+------+--------------+--------------------+
|  上海|  三亚|上海航空FM9206|[{'qry_dt': '2020...|
|  上海|  三亚|上海航空FM9238|[{'qry_dt': '2019...|
|  上海|  三亚|上海航空FM9518|[{'qry_dt': '2019...|
+------+------+--------------+--------------------+
only showing top 3 rows



## 1.城市间，无航线，最低价格列表

### 出发城市—到达城市—最低价格列表<时间，价格，航空公司>

In [3]:
df_c = df.collect()

In [4]:
len(df_c)

18376

In [5]:
df_c[0][1],df_c[0][2],df_c[0][3]

('三亚',
 '上海航空FM9206',
 "[{'qry_dt': '2020-01-09', 'dep_dt': '2020-01-10', 'price': '980'}, {'qry_dt': '2020-01-25', 'dep_dt': '2020-01-26', 'price': '1020'}, {'qry_dt': '2020-03-10', 'dep_dt': '2020-03-18', 'price': '310'}, {'qry_dt': '2020-03-22', 'dep_dt': '2020-03-30', 'price': '300'}]")

In [6]:
def deal():
    Dict = {}
    for i in range(len(df_c)):   # 遍历航班
        arr_ct = df_c[i][0]
        dep_ct = df_c[i][1]
        line = df_c[i][2]
        price_list = df_c[i][3]
        s_js = eval(price_list)        # price_list 字符串转json
        if len(s_js) == 0 :
            continue
        for j in range(len(s_js)):      # 遍历price_list
            price = int(s_js[j]['price'])
            if (arr_ct,dep_ct) not in Dict :    # 城市对尚未建立
                Dict[(arr_ct,dep_ct)] = {s_js[j]['dep_dt']:(price,line)}
            else :
                if s_js[j]['dep_dt'] not in Dict[(arr_ct,dep_ct)] :
                    Dict[(arr_ct,dep_ct)][s_js[j]['dep_dt']] = (price,line)
                elif Dict[(arr_ct,dep_ct)][s_js[j]['dep_dt']][0] > price:
                    Dict[(arr_ct,dep_ct)][s_js[j]['dep_dt']] = (price,line)
    return Dict

dct = deal()

#### 按出发时间排序

In [7]:
for a in dct :
    dct[a] = sorted(dct[a].items(),key=lambda item:item[0], reverse=False)

#### 整理格式

In [8]:
def toDB(Dict):
    final_Dict = []
    for key in Dict :
        td = {}
        td['arr_ct'] = key[0]
        td['dep_ct'] = key[1]
        # print(key)
        tl = []
        for s in range(len(Dict[key])):
            # print(Dict[key])
            st = Dict[key][s][0] + '|' + f"{Dict[key][s][1][0]}|{Dict[key][s][1][1]}"
            tl.append(st)
        td["min_price_list"] = f'{tl}'
        final_Dict.append(td)
    return final_Dict
Ans = toDB(dct)
Ans[0]

{'arr_ct': '上海',
 'dep_ct': '三亚',
 'min_price_list': "['2019-04-15|1190|吉祥航空HO1128', '2019-04-16|990|金鹏航空Y87502', '2019-04-17|950|吉祥航空HO1128', '2019-04-18|1070|吉祥航空HO1128', '2019-04-19|950|吉祥航空HO1128', '2019-04-20|1100|春秋航空9C8802', '2019-04-21|1130|南方航空CZ6753', '2019-04-22|1070|吉祥航空HO1128', '2019-04-23|940|上海航空FM9522', '2019-04-24|850|吉祥航空HO1128', '2019-04-25|784|金鹏航空Y87502', '2019-04-26|850|吉祥航空HO1128', '2019-04-27|850|吉祥航空HO1128', '2019-04-28|850|吉祥航空HO1128', '2019-04-29|784|金鹏航空Y87502', '2019-04-30|784|金鹏航空Y87502', '2019-05-01|800|春秋航空9C8802', '2019-05-02|880|南方航空CZ6753', '2019-05-03|1190|吉祥航空HO1128', '2019-05-04|2180|上海航空FM9518', '2019-05-05|1190|吉祥航空HO1128', '2019-05-06|860|春秋航空9C8802', '2019-05-07|750|春秋航空9C8802', '2019-05-08|600|春秋航空9C8802', '2019-05-09|600|春秋航空9C8802', '2019-05-10|600|春秋航空9C8802', '2019-05-11|600|南方航空CZ6753', '2019-05-12|700|春秋航空9C8802', '2019-05-13|750|南方航空CZ6781', '2019-05-14|650|东方航空MU5467', '2019-05-15|540|春秋航空9C8802', '2019-05-16|490|金鹏航空Y87502', '2019-05-

In [9]:
len(Ans)

1419

### 写数据库(CtoC_min_price)

In [11]:
json_str = json.dumps(Ans,ensure_ascii=False)
with open('tmp_minprice_toDB.json', 'w',encoding='utf-8') as json_file:
    json_file.write(json_str)

In [14]:
ds = spark.read.json('tmp_minprice_toDB.json')
ds = ds.select("dep_ct","arr_ct","min_price_list")
ds.show(5)

+------+--------+--------------------+
|dep_ct|  arr_ct|      min_price_list|
+------+--------+--------------------+
|  三亚|    上海|['2019-04-15|1190...|
|  三亚|乌鲁木齐|['2019-04-15|1750...|
|  三亚|    兰州|['2019-04-16|1300...|
|  三亚|    北京|['2019-04-15|1560...|
|  三亚|    南京|['2019-04-15|1175...|
+------+--------+--------------------+
only showing top 5 rows



In [15]:
prop = {
    'user':'root',
    'password':'1',
    'driver':'com.mysql.jdbc.Driver'
}
url = 'jdbc:mysql://localhost:3306/flight?useUnicode=true&characterEncoding=utf8'

In [17]:
ds.write.jdbc(url, 'CtoC_min_price', properties=prop)

## 2.城市间航班，历史价格列表

### 出发城市—到达城市—航班—历史价格列表

In [18]:
df.show(5)

+------+------+--------------+--------------------+
|arr_ct|dep_ct|          line|          price_list|
+------+------+--------------+--------------------+
|  上海|  三亚|上海航空FM9206|[{'qry_dt': '2020...|
|  上海|  三亚|上海航空FM9238|[{'qry_dt': '2019...|
|  上海|  三亚|上海航空FM9518|[{'qry_dt': '2019...|
|  上海|  三亚|上海航空FM9522|[{'qry_dt': '2019...|
|  上海|  三亚|上海航空FM9538|[{'qry_dt': '2019...|
+------+------+--------------+--------------------+
only showing top 5 rows



In [19]:
len(df_c)

18376

In [22]:
def deal_2():
    Dict = {}
    for i in range(len(df_c)):   # 遍历航班
        arr_ct = df_c[i][0]
        dep_ct = df_c[i][1]
        line = df_c[i][2]
        price_list = df_c[i][3]
        s_js = eval(price_list)        # price_list 字符串转json
        if len(s_js) == 0 :
            continue
        for j in range(len(s_js)):      # 遍历price_list
            price = int(s_js[j]['price'])
            if (arr_ct,dep_ct,line) not in Dict :    # 城市对尚未建立
                Dict[(arr_ct,dep_ct,line)] = {s_js[j]['dep_dt']:price}
            else :
                if s_js[j]['dep_dt'] not in Dict[(arr_ct,dep_ct,line)] :
                    Dict[(arr_ct,dep_ct,line)][s_js[j]['dep_dt']] = price
                elif Dict[(arr_ct,dep_ct,line)][s_js[j]['dep_dt']] > price:
                    Dict[(arr_ct,dep_ct,line)][s_js[j]['dep_dt']] = price
    return Dict

dct_2 = deal_2()

In [23]:
for a in dct_2 :
    dct_2[a] = sorted(dct_2[a].items(),key=lambda item:item[0], reverse=False)

In [25]:
len(dct_2[('上海','三亚','上海航空FM9238')])

81

In [26]:
len(dct_2)

18376

In [30]:
def toDB(Dict):
    final_Dict = []
    for key in Dict :
        td = {}
        td['arr_ct'] = key[0]
        td['dep_ct'] = key[1]
        td['line'] = key[2]
        # print(key)
        tl = []
        for s in range(len(Dict[key])):
            # print(Dict[key])
            st = Dict[key][s][0] + '|' + f"{Dict[key][s][1]}"
            tl.append(st)
        td["price_list"] = f'{tl}'
        final_Dict.append(td)
    return final_Dict
Ans_2 = toDB(dct_2)
Ans_2[0]

{'arr_ct': '上海',
 'dep_ct': '三亚',
 'line': '上海航空FM9206',
 'price_list': "['2020-01-10|980', '2020-01-26|1020', '2020-03-18|310', '2020-03-30|300']"}

In [31]:
Ans_2[1]

{'arr_ct': '上海',
 'dep_ct': '三亚',
 'line': '上海航空FM9238',
 'price_list': "['2019-10-27|180', '2019-10-28|160', '2019-10-29|160', '2019-10-30|160', '2019-11-01|160', '2019-11-02|160', '2019-11-03|350', '2019-11-04|160', '2019-11-05|550', '2019-11-06|160', '2019-11-08|160', '2019-11-09|240', '2019-11-10|450', '2019-11-11|240', '2019-11-12|160', '2019-11-13|160', '2019-11-15|160', '2019-11-16|550', '2019-11-17|550', '2019-11-18|550', '2019-11-19|160', '2019-11-20|160', '2019-11-22|180', '2019-11-23|550', '2019-11-24|550', '2019-11-25|550', '2019-11-26|235', '2019-11-27|160', '2019-11-29|180', '2019-11-30|605', '2019-12-01|605', '2019-12-02|210', '2019-12-03|175', '2019-12-04|175', '2019-12-06|295', '2019-12-09|295', '2019-12-10|175', '2019-12-11|175', '2019-12-13|210', '2019-12-14|1115', '2019-12-15|875', '2019-12-16|590', '2019-12-17|295', '2019-12-18|200', '2019-12-20|580', '2019-12-21|340', '2019-12-22|550', '2019-12-23|200', '2019-12-24|190', '2019-12-25|180', '2019-12-27|400', '2019-1

### 写数据库(CtoC_Flight_price)

In [33]:
json_str_2 = json.dumps(Ans_2,ensure_ascii=False)
with open('tmp_flightprice_toDB.json', 'w',encoding='utf-8') as json_file:
    json_file.write(json_str_2)

In [34]:
dtt = spark.read.json('tmp_flightprice_toDB.json')
dtt.select("dep_ct","arr_ct","line","price_list")
dtt.show(3)

+------+------+--------------+--------------------+
|arr_ct|dep_ct|          line|          price_list|
+------+------+--------------+--------------------+
|  上海|  三亚|上海航空FM9206|['2020-01-10|980'...|
|  上海|  三亚|上海航空FM9238|['2019-10-27|180'...|
|  上海|  三亚|上海航空FM9518|['2019-04-15|1290...|
+------+------+--------------+--------------------+
only showing top 3 rows



In [35]:
prop = {
    'user':'root',
    'password':'1',
    'driver':'com.mysql.jdbc.Driver'
}
url = 'jdbc:mysql://localhost:3306/flight?useUnicode=true&characterEncoding=utf8'

In [36]:
dtt.write.jdbc(url, 'CtoC_Flight_price', properties=prop)